# c2score_scoring_workbench_data

JavaScript

In [3]:
import * as tslab from 'tslab';
import fetch from 'node-fetch';

In [4]:
const GRAPHL_API = "http://54.166.100.68:8080/v1/graphql";

In [5]:
async function getData() {
    
    const body =  JSON.stringify({
      query: 
        `{
          c2score_scoring_workbench_data_cached(
              where: {agedays: {_gt: "90"}, 
                      _and: {annreturn: {_gt: "0"}, 
                      _and: {deltaequity180days: {_gt: "0"}}}}, 
                      order_by: {strategyid: asc}) 
          {
            strategyid
            strategyname
            added
            agedays
            annreturn
            dailymaxlevmax
            deltaequity90days
            maxdrawdownpcnt
            maxworstlosspercentequity060
            numtrades
            optionpercent
            pcntmonthsprofitable
            shortoptionscovered
            tradedaysago
          }
        }`
    });

    const response = await fetch(
        GRAPHL_API,
        {
          method: 'post',
          body: body,
          headers: {'Content-Type': 'application/json'},
        }
      );

  let json = await response.json();
  return json.data.c2score_scoring_workbench_data_cached;
}


const resultArray = await getData();
console.log(resultArray[0]);


{
  strategyid: 13202557,
  strategyname: 'extreme-os',
  added: '2005-02-17',
  agedays: 6124.93017578125,
  annreturn: 0.2791000008583069,
  dailymaxlevmax: 2.7200000286102295,
  deltaequity90days: 12.029999732971191,
  maxdrawdownpcnt: 63.63999938964844,
  maxworstlosspercentequity060: 13.47,
  numtrades: 4230,
  optionpercent: 0,
  pcntmonthsprofitable: 69.80000305175781,
  shortoptionscovered: 100,
  tradedaysago: 1
}


In [6]:
function riskFun(strat) {
    const result = 
      (20 - strat.maxdrawdownpcnt * (20/45))
    + (strat.maxworstlosspercentequity060 === null ? 0 : (20 - strat.maxworstlosspercentequity060 * (20.0/8)))
    + (10 - strat.optionpercent * (10/1))
    + (strat.shortoptionscovered >= 100 ? 10 : strat.shortoptionscovered / 100 * 10);
    return result;
}

function behavioralFun(strat) {
    const result = 
      (strat.agedays >= 720 ? 40 : strat.agedays/720 * 40)
    + (20 - strat.dailymaxlevmax * (20/10)) 
    + (strat.tradedaysago <= 60 ? 20 : 0)
    + (strat.pcntmonthsprofitable >= 100 ? 5 : strat.pcntmonthsprofitable / 100 * 5)
    + (strat.numtrades >= 100 ? 20 : strat.numtrades / 100 * 20)
    + ((strat.deltaequity90days <= 0 && strat.deltaequity90days > 7) ? 5 : 0); // <======= ERROR! WRONG CONDITION
    return result;
}

function profitabilityFun(strat) {
    const result = strat.annreturn > 0.6 ? 20 :  strat.annreturn / 0.6 * 20;
    return result;
}


let data = resultArray
//  done in GraphQL: .filter(strat => !(strat.AgeDays < 90 || strat.AnnReturn < 0 || strat.DeltaEquity180Days < 0))
            .map(strat => {
                return {...strat, 
                        ...{scoreRisk:riskFun(strat)},
                        ...{scoreBehavior: behavioralFun(strat)},
                        ...{scoreProfitability: profitabilityFun(strat)},
                        ...{score: (riskFun(strat) + behavioralFun(strat) + profitabilityFun(strat))}
                       }})
            .sort((a,b) => b.score - a.score );

console.log(data[0]);

{
  strategyid: 126454200,
  strategyname: 'GardCap Discretionary',
  added: '2019-12-03',
  agedays: 723.1599731445312,
  annreturn: 0.1446000039577484,
  dailymaxlevmax: 1.850000023841858,
  deltaequity90days: 3.4100000858306885,
  maxdrawdownpcnt: 3.6600000858306885,
  maxworstlosspercentequity060: 1.14,
  numtrades: 186,
  optionpercent: 0,
  pcntmonthsprofitable: 70.80000305175781,
  shortoptionscovered: 100,
  tradedaysago: 0.800000011920929,
  scoreRisk: 55.52333329518636,
  scoreBehavior: 99.84000010490418,
  scoreProfitability: 4.820000131924948,
  score: 160.1833335320155
}


In [7]:
let uidata = data
            .map(i => ( 
                  {  
                      Score: i.score.toFixed(2), 
                      StrategyId: i.strategyid,  
                      StrategyName: i.strategyname, 
                      scoreRisk: i.scoreRisk.toFixed(2),
                      scoreBehavior: i.scoreBehavior.toFixed(2),
                      scoreProfitability: i.scoreProfitability.toFixed(2)                      
                  })
                )            ;
console.table(uidata.slice(0,20));

┌─────────┬──────────┬────────────┬────────────────────────────┬───────────┬───────────────┬────────────────────┐
│ (index) │  Score   │ StrategyId │        StrategyName        │ scoreRisk │ scoreBehavior │ scoreProfitability │
├─────────┼──────────┼────────────┼────────────────────────────┼───────────┼───────────────┼────────────────────┤
│    0    │ '160.18' │ 126454200  │  'GardCap Discretionary'   │  '55.52'  │    '99.84'    │       '4.82'       │
│    1    │ '158.12' │ 117734561  │       'TQQQ Aspire'        │  '44.70'  │    '94.58'    │      '18.85'       │
│    2    │ '156.89' │ 124696549  │     '4Timing Trend ML'     │  '53.34'  │    '99.55'    │       '4.01'       │
│    3    │ '156.87' │ 128743489  │  'JWB Stock and ETF ORB'   │  '49.24'  │    '88.73'    │      '18.91'       │
│    4    │ '153.17' │ 126054331  │       'CkNN Algo V'        │  '35.53'  │    '97.64'    │      '20.00'       │
│    5    │ '151.99' │ 126444124  │      'Saivers US ETF'      │  '38.39'  │    '97.90' 